In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as sp
import seaborn as sns

import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
from pandas.plotting import autocorrelation_plot

import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import pacf

from mytsfunctions import *

In [ ]:
my_create_large_images()

In [ ]:
df0 = pd.read_csv("us_emp_retail.csv")
#df0 = pd.read_csv("../case-study/egypt_exports_gdp.csv")
df0.head()

In [ ]:
df =  df0['Employed']
#df =  df0['Exports_percent_GDP']
df.head(5)

In [ ]:
df.plot()

In [ ]:
sm.tsa.stattools.adfuller(df)

In [ ]:
sm.tsa.stattools.kpss(df)

In [ ]:
#tsr = seasonal_decompose(df,model='additive',period=12)
#tsr.plot()

In [ ]:
# Creating STL decomposition

In [ ]:
from statsmodels.tsa.seasonal import STL
stl = STL(df, period=12)

In [ ]:
res = stl.fit()
fig = res.plot()

In [ ]:
fig = plt.figure(figsize=(14, 8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(df, lags=50, ax=ax1)

ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(df, lags=50, ax=ax2)

In [ ]:
# detrend, if required, and plot
# dtrend = difference(df,1)
# #dtrend = df
# plt.plot(dtrend)

In [ ]:
diff1 = difference(df,1)
diff1_diff12 = difference(diff1,12)

In [ ]:
plt.plot(diff1)

In [ ]:
plt.plot(diff1_diff12)

In [ ]:
fig = plt.figure(figsize=(14, 8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(diff1_diff12, lags=14, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(diff1_diff12, lags=14, ax=ax2)

In [ ]:
#sm.tsa.stattools.arma_order_select_ic(df)

In [ ]:
# First de-trend, then de-seasonalize? Or first de-seasonalize, then de-trend?

In [ ]:
tdf = df
tdf_diff1 = difference(tdf,1)
tdf_diff12 = difference(tdf,12)

tdf_diff1_diff12 = difference(tdf_diff1,12)
tdf_diff12_diff1 = difference(tdf_diff12,1)

In [ ]:
#plt.plot(tdf_diff12)

In [ ]:
fig = plt.figure(figsize=(14, 8))
fig.add_subplot(211)
plt.plot(tdf_diff1_diff12)

fig.add_subplot(212)
plt.plot(tdf_diff12_diff1)

In [ ]:
sm.tsa.stattools.adfuller(tdf_diff1_diff12)

In [ ]:
# Check the statistical behaviour of original and stationary data (mean, sd)
len(tdf)

In [ ]:
# for i in [0,50,100,150,200]:
#     sidx = i
#     eidx = i+50
#     print("******")
#     print(f"Original Data: Mean : {np.mean(tdf[sidx:eidx])}, SD : {np.std(tdf[sidx:eidx])}")
#     #print(f"Diff data :    Mean : {np.mean(tdf_diff1[sidx:eidx])}, SD : {np.std(tdf_diff1[sidx:eidx])}")
#     print(f"Diff data :    Mean : {np.mean(tdf_diff1_diff12[sidx:eidx])}, SD : {np.std(tdf_diff1_diff12[sidx:eidx])}")

In [ ]:
#runMyAR1(tdf)

In [ ]:
#runMyAR1(tdf_diff1)
#runMyAR1(tdf_diff1_diff12)

In [ ]:
# Perform ARIMA

In [ ]:
my_model = sm.tsa.arima.ARIMA(df,order=(3,1,4),seasonal_order=(0,1,0,12))
#my_model = sm.tsa.arima.ARIMA(df,order=(3,1,0))
my_model_res = my_model.fit()
print(my_model_res.summary())

In [ ]:
pred = my_model_res.plot_diagnostics()

In [ ]:
tforecast =  my_model_res.forecast(12)

In [ ]:
plt.plot(tdf)
plt.plot(tforecast)

In [ ]:
tpredict =  my_model_res.predict(40)
#plt.plot(tpredict)

In [ ]:
plt.plot(tdf)
plt.plot(tpredict)
plt.plot(tforecast)

In [ ]:
tforecast2 = my_model_res.get_forecast(12)
confint = np.array(tforecast2.conf_int())

In [ ]:
fig,ax = plt.subplots(figsize=(16,12))
ax.plot(df.index, df, label='data')
ax.plot(tforecast2.predicted_mean.index, tforecast2.predicted_mean, label='forecast')
ax.fill_between(tforecast2.predicted_mean.index, confint[:,0], confint[:,1],color='gray',alpha=0.3)
ax.legend()